<a href="https://colab.research.google.com/github/shahriariit/COVID-19-Twitter-Data-Analysis/blob/master/TM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
dataset=pd.read_csv("https://raw.githubusercontent.com/susanli2016/Machine-Learning-with-Python/master/dataset.csv")

In [0]:
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [3]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
     return WordNetLemmatizer().lemmatize(word)

In [6]:
for w in ['dogs', 'ran', 'discouraged']:
    print(w, get_lemma(w), get_lemma2(w))

dogs dog dog
ran run ran
discouraged discourage discouraged


In [7]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [16]:
import random
text_data = []
with open('dataset.csv') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        if random.random() > .99:
            print(tokens)
            text_data.append(tokens)

['enchant', 'scissors', 'scissor', 'interface', 'support', 'cutting', 'interactive', 'fabrication']
['generalize', 'arbitrary', 'resize', 'video', 'transcoding']
['proposal', 'blind', 'equalizer', 'using', 'output', 'signal', 'decision', 'device']
['frequency', 'response', 'masking', 'base', 'reconfigurable', 'channel', 'filter', 'software', 'radio', 'receiver']
['piezoelectric', 'actuator', 'driver', 'circuit', 'automatic', 'focusing', 'mobile', 'phone', 'camera']
['definition', 'evaluation', 'access', 'rule', 'management', 'system']
['generic', 'solution', 'warehousing', 'business', 'process']
['normalization', 'optimization', 'schema', 'mapping']
['locating', 'source', 'large', 'distribute', 'system']
['bilateral', 'filter', 'display', 'dynamic', 'range', 'image']
['selectivity', 'estimation', 'extensible', 'database', 'neural', 'network', 'approach']
['double', 'insertion', 'enhance', 'architecture', 'clock', 'route', 'reliability']
['annotating', 'search', 'table', 'using', 'entit

In [0]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)

In [0]:
corpus = [dictionary.doc2bow(text) for text in text_data]

In [19]:
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [20]:
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
